In [1]:
import numpy as np
import pandas as pd
from nltk import tokenize
import nltk
from string import punctuation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import unidecode
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ramon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
dataset_original = pd.read_csv("./datasets/movies_amostra.csv")

In [3]:
len(dataset_original)
dataset = dataset_original.copy()

In [4]:
dataset.isna().any()

id                   False
titulo               False
adulto               False
orcamento            False
idioma_original       True
popularidade         False
data_de_estreia      False
resumo                True
receita              False
duracao               True
genero               False
ator_1                True
ator_2                True
ator_3                True
ator_4                True
ator_5                True
dirigido_por          True
escrito_por_1         True
escrito_por_2         True
historia_original     True
dtype: bool

In [28]:
dataset_original.columns

Index(['id', 'titulo', 'adulto', 'orcamento', 'idioma_original',
       'popularidade', 'data_de_estreia', 'resumo', 'receita', 'duracao',
       'genero', 'ator_1', 'ator_2', 'ator_3', 'ator_4', 'ator_5',
       'dirigido_por', 'escrito_por_1', 'escrito_por_2', 'historia_original'],
      dtype='object')

In [27]:
dataset_original["idioma_original"].value_counts()

en    2132
fr     152
it     128
ja      98
cn      65
ru      55
es      54
de      51
zh      44
hi      39
sv      27
ko      25
fi      22
pl      14
nl      11
tr      10
da      10
pt      10
ta       8
th       6
no       5
hu       4
el       4
sr       4
ml       4
ro       3
te       2
cs       2
he       2
lb       1
uk       1
et       1
ur       1
xx       1
mr       1
fa       1
id       1
Name: idioma_original, dtype: int64

In [5]:
dataset.drop(["idioma_original", "data_de_estreia", "historia_original", "duracao"], inplace=True, axis=1)

In [6]:
dataset.head()

,id,titulo,adulto,orcamento,popularidade,resumo,receita,genero,ator_1,ator_2,ator_3,ator_4,ator_5,dirigido_por,escrito_por_1,escrito_por_2
0,86295,Treasure of the Yankee Zephyr,False,0,2.0,In a lake high in the mountains of New Zealand...,0.0,Action,Ken Wahl,Lesley Ann Warren,Donald Pleasence,George Peppard,Bruno Lawrence,David Hemmings,Everett De Roche,NaN
1,289198,Redeemer,False,0,2.0,A former hit-man for a drug cartel becomes a v...,0.0,Action,Marko Zaror,Loreto Aravena,Mauricio Diocares,José Luís Mósca,Noah Segan,Ernesto Díaz Espinoza,NaN,NaN
2,24382,Big Deal on Madonna Street,False,0,11.0,"Peppe, formerly a boxer, organizes the break-i...",0.0,Comedy,Vittorio Gassman,Renato Salvatori,Memmo Carotenuto,Rossana Rory,Carla Gravina,Mario Monicelli,Agenore Incrocci,Furio Scarpelli
3,479,Shaft,False,46000000,13.0,New York police detective John Shaft arrests W...,107196498.0,Action,Samuel L. Jackson,Jeffrey Wright,Christian Bale,Busta Rhymes,Dan Hedaya,John Singleton,NaN,NaN
4,37712,Pistol Opera,False,0,2.0,"As one of Suzuki's last fims, it is related to...",0.0,Action,Makiko Esumi,Sayoko Yamaguchi,Hanae Kan,Masatoshi Nagase,Mikijiro Hira,Seijun Suzuki,Kazunori Ito,NaN


In [7]:
dataset["classificacao"] = dataset["genero"].replace(["Action", "Comedy"],[0, 1])
dataset.drop(0, axis=0, inplace=True)

In [8]:
print(dataset["classificacao"].value_counts())

1    1988
0    1012
Name: classificacao, dtype: int64


In [9]:
dataset["resumo"].isna().sum()

69

In [10]:
dataset = dataset[dataset['resumo'].notna()]

In [11]:
dataset["resumo"].isna().sum()

0

In [12]:
resumos = dataset["resumo"]

In [13]:
dataset.fillna("nenhum", inplace=True)

In [14]:
todos_atores = np.array(dataset[["ator_1", "ator_2", "ator_3", "ator_4", "ator_5"]]).reshape(-1, 1)
print(len(todos_atores))
print(len(np.unique(todos_atores)))
label_atores = LabelEncoder().fit(todos_atores)
cat_atores = label_atores.transform(todos_atores)
encoded_atores = list()
for index, row in dataset.iterrows():
    labels_row = label_atores.transform([row['ator_1'], row['ator_2'], row['ator_3'], row['ator_4'], row['ator_5']])
    zeros = np.zeros(len(np.unique(todos_atores)))
    zeros[labels_row] = 1
    encoded_atores.append(zeros)
dataset["atores_encoded"] = encoded_atores
dataset.drop(["ator_1", "ator_2", "ator_3", "ator_4", "ator_5"], inplace=True, axis=1)

14655
9503


c:\users\ramon\anaconda3\envs\mlcefet\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [15]:
label_dirigido = LabelEncoder().fit(dataset["dirigido_por"])
labelled_dirigido = label_dirigido.transform(dataset["dirigido_por"]).reshape(-1, 1)
onehot_dirigido = OneHotEncoder(sparse=False).fit(labelled_dirigido)
onehotted_dirigido = onehot_dirigido.transform(labelled_dirigido)
dataset["dirigido_encoded"] = onehotted_dirigido.tolist()
dataset.drop("dirigido_por", inplace=True, axis=1)

In [16]:
todos_escritores = np.array(dataset[["escrito_por_1", "escrito_por_2"]]).reshape(-1, 1)
print(len(todos_escritores))
print(len(np.unique(todos_escritores)))
label_escritores = LabelEncoder().fit(todos_escritores)
cat_escritores = label_escritores.transform(todos_escritores)
encoded_escritores = list()
for index, row in dataset.iterrows():
    labels_row = label_escritores.transform([row['escrito_por_1'], row['escrito_por_2']])
    zeros = np.zeros(len(np.unique(todos_escritores)))
    zeros[labels_row] = 1
    encoded_escritores.append(zeros)
dataset["escritores_encoded"] = encoded_escritores
dataset.drop(["escrito_por_1", "escrito_por_2"], axis=1, inplace=True)

5862
1424


c:\users\ramon\anaconda3\envs\mlcefet\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [17]:
escalados = StandardScaler().fit_transform(dataset[["receita", "popularidade", "orcamento"]])
dataset["receita_scaled"] = escalados[:, 0]
dataset["popularidade_scaled"] = escalados[:, 1]
dataset["orcamento_scaled"] = escalados[:, 2]
dataset.drop(["receita", "orcamento", "popularidade"], axis=1, inplace=True)

In [18]:
#Tratamento do texto
pontuacao = [i for i in punctuation]
tokenizer_pontuacao = tokenize.WordPunctTokenizer()
pontuacao_stopwords = pontuacao + nltk.corpus.stopwords.words("english")
stopwords_sem_acento = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]
stemmer = SnowballStemmer("english")

dataset["resumo_tratado"] = [unidecode.unidecode(texto) for texto in dataset["resumo"]]
frase_processada = list()
for resumo in dataset["resumo_tratado"]:
    nova_frase = list()
    resumo = resumo.lower()
    palavras_texto = tokenizer_pontuacao.tokenize(resumo)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))
dataset["resumo_tratado"] = frase_processada

In [19]:
vetorizar = TfidfVectorizer(lowercase=False)
bag_of_words = vetorizar.fit_transform(dataset["resumo_tratado"])
dataset["bag_of_words"] = bag_of_words

In [20]:
dataset.head()

,id,titulo,adulto,resumo,classificacao,atores_encoded,dirigido_encoded,escritores_encoded,receita_scaled,popularidade_scaled,orcamento_scaled,resumo_tratado,bag_of_words
0,86295,Treasure of the Yankee Zephyr,False,In a lake high in the mountains of New Zealand...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.240121,-0.432328,-0.308391,lake high mountain new zealand hunter gibbi gi...,"(0, 12245)\t0.2137935761906109\n (0, 7150)\..."
1,289198,Redeemer,False,A former hit-man for a drug cartel becomes a v...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.240121,-0.432328,-0.308391,former hit man drug cartel becom vigilant pay ...,"(0, 12245)\t0.2137935761906109\n (0, 7150)\..."
2,24382,Big Deal on Madonna Street,False,"Peppe, formerly a boxer, organizes the break-i...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.240121,0.496670,-0.308391,pepp former boxer organ break pawnshop tiberio...,"(0, 12245)\t0.2137935761906109\n (0, 7150)\..."
3,479,Shaft,False,New York police detective John Shaft arrests W...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.004307,0.703114,1.468008,new york polic detect john shaft arrest walter...,"(0, 12245)\t0.2137935761906109\n (0, 7150)\..."
4,37712,Pistol Opera,False,"As one of Suzuki's last fims, it is related to...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.240121,-0.432328,-0.308391,one suzuki last fim relat 1967 brand kill eith...,"(0, 12245)\t0.2137935761906109\n (0, 7150)\..."


In [21]:
inputs = dataset[["atores_encoded", "dirigido_encoded", 
                      "escritores_encoded", "receita_scaled", 
                      "popularidade_scaled", "orcamento_scaled", "bag_of_words"]]

In [22]:
model_input = list()
for index, row in inputs.iterrows():
    model_input.append(np.hstack((row["atores_encoded"], row["dirigido_encoded"], row["escritores_encoded"], row["receita_scaled"], 
                      row["popularidade_scaled"], row["orcamento_scaled"])))

In [23]:
# Testando treinar com os parâmetros atores, diretor, escritores, receita, popularidade e orçamento
treino, teste, classe_treino, classe_teste = train_test_split(model_input, dataset["classificacao"], random_state=1)
regressao_logistica = LogisticRegression(max_iter=1000)
regressao_logistica.fit(treino, classe_treino)
print("Regressao Logistica", regressao_logistica.score(teste, classe_teste))
random_forest = RandomForestClassifier()
random_forest.fit(treino, classe_treino)
print("Random Forest", random_forest.score(teste, classe_teste))
rede_neural = MLPClassifier()
rede_neural.fit(treino, classe_treino)
print("Rede Neural", rede_neural.score(teste, classe_teste))

Regressao Logistica 0.7353342428376535
Random Forest 0.6807639836289222
Rede Neural 0.7980900409276944


In [24]:
# Testando treinar apenas com o resumo
treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, dataset["classificacao"], random_state=1)
regressao_logistica = LogisticRegression(max_iter=1000)
regressao_logistica.fit(treino, classe_treino)
print("Regressao Logistica", regressao_logistica.score(teste, classe_teste))
random_forest = RandomForestClassifier()
random_forest.fit(treino, classe_treino)
print("Random Forest", random_forest.score(teste, classe_teste))
rede_neural = MLPClassifier()
rede_neural.fit(treino, classe_treino)
print("Rede Neural", rede_neural.score(teste, classe_teste))

Regressao Logistica 0.7517053206002728
Random Forest 0.7680763983628922
Rede Neural 0.7885402455661664


In [25]:
# #Expandir a bag of words ocupa memória demais, procurar outra forma de fazer isso
# full_input = list()
# for index, row in inputs.iterrows():
#     full_input.append(np.hstack((row["atores_encoded"], row["dirigido_encoded"], row["escritores_encoded"], row["receita_scaled"], 
#                       row["popularidade_scaled"], row["orcamento_scaled"], row["bag_of_words"].toarray().reshape(-1))))

# #Testar treinar com todas as entradas
# treino, teste, classe_treino, classe_teste = train_test_split(full_input, dataset["classificacao"], random_state=1)
# regressao_logistica = LogisticRegression(max_iter=1000)
# regressao_logistica.fit(treino, classe_treino)
# print("Regressao Logistica", regressao_logistica.score(teste, classe_teste))
# random_forest = RandomForestClassifier()
# random_forest.fit(treino, classe_treino)
# print("Random Forest", random_forest.score(teste, classe_teste))
# rede_neural = MLPClassifier()
# rede_neural.fit(treino, classe_treino)
# print("Rede Neural", rede_neural.score(teste, classe_teste))